In [1]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import re
import difflib
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
import warnings 
warnings.filterwarnings("ignore")

In [2]:
xml_file = "s3994666_dataset1.xml"

tree = ET.parse(xml_file)
root = tree.getroot()

#
data = []

for ad in root.findall('.//Ad'):
    add_data = {
        'Id': ad.find('Id').text if ad.find('Id') is not None else 'non-specified',
        'Title': ad.find('Title').text if ad.find('Title') is not None and ad.find('Title').text != 'N/A' else 'non-specified',
        'Location': ad.find('Location').text if ad.find('Location') is not None and ad.find('Location').text != 'N/A' else 'non-specified',
        'Company': ad.find('Company').text if ad.find('Company') is not None and ad.find('Company').text != 'N/A' else 'non-specified',
        'ContractType': ad.find('ContractType').text if ad.find('ContractType') is not None and ad.find('ContractType').text != 'N/A' else 'non-specified',
        'ContractTime': ad.find('ContractTime').text if ad.find('ContractTime') is not None and ad.find('ContractTime').text != 'N/A' else 'non-specified',
        'Category': ad.find('Category').text if ad.find('Category') is not None else 'non-specified',
        'Salary': ad.find('Salary').text if ad.find('Salary') is not None else 'non-specified',
        'OpenDate': ad.find('.//OpenDate').text if ad.find('.//OpenDate') is not None else 'non-specified',
        'CloseDate': ad.find('.//CloseDate').text if ad.find('.//CloseDate') is not None else 'non-specified',
        'SourceName': root.find('.//SourceName').text if root.find('.//SourceName') is not None and root.find('.//SourceName').text != 'N/A' else 'non-specified'
    }
    data.append(add_data)

# Create DataFrame
df = pd.DataFrame(data)

# Display the head of the DataFrame to check the result
df.head(n=10)


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,70215060,Sales Bid Coordinator,Surrey,CBSbutler,non-specified,permanent,Engineering Jobs,24000,20130301T000000,20130530T000000,cv-library.co.uk
1,71749951,"Primary Teachers Nailsea, Yatton, Congresbury",Weston-Super-Mare,Hays Specialist Recruitment Further Education,non-specified,permanent,Teaching Jobs,18720,20120804T120000,20120903T120000,cv-library.co.uk
2,69080174,Internal Technical Engineer Heat Pumps,Derbyshire,Bennett and Game Recruitment LTD,non-specified,permanent,Engineering Jobs,19000,20130224T150000,20130525T150000,cv-library.co.uk
3,67638009,Regional Sales Manager : PPE and Gas Detectors,UK,On Target Recruitment Ltd,non-specified,permanent,Sales Jobs,42500,20130222T150000,20130423T150000,cv-library.co.uk
4,68217496,Senior Assembly Engineer,Durham,Baltic Recruitment Services Ltd,non-specified,permanent,Engineering Jobs,38750,20131012T120000,20131111T120000,cv-library.co.uk
5,69967015,Associate Civil Infrastructure Design Engineer,Essex,Energi Recruitment Services Ltd,non-specified,permanent,Engineering Jobs,45000,20131128T000000,20131228T000000,cv-library.co.uk
6,66600487,Sales Advisor **** an hour Immediate Start,Sittingbourne,Huntress Group,non-specified,contract,Sales Jobs,13440,20120508T150000,20120607T150000,cv-library.co.uk
7,66925309,"Service Desk Analyst Active Directory, Citrix...",London,Pearson Whiffin Affinity,non-specified,permanent,IT Jobs,23500,20121003T150000,20130101T150000,cv-library.co.uk
8,69001320,Field Sales (Home based),Birmingham,Parkside,non-specified,permanent,Sales Jobs,27500,20130313T000000,20130611T000000,cv-library.co.uk
9,70216327,Risk Manager (Junior/Senior),Bristol,Electus Recruitment Solutions,non-specified,permanent,Engineering Jobs,45000,20121009T150000,20121208T150000,cv-library.co.uk


In [3]:
error_list_columns = ["indexOfdf", "Id", "ColumnName", "Original", "Modified", "ErrorType", "Fixing"]
error_list_df = pd.DataFrame(columns=error_list_columns)

In [4]:
IDS= df.Id.value_counts()
print(df.Id.head())

0    70215060
1    71749951
2    69080174
3    67638009
4    68217496
Name: Id, dtype: object


In [5]:
# Checking value counts of the 'Title' column
title_counts = df['Title'].value_counts()
title_counts


Sales Executive Bedford                                              2
RGN/ Ward Nurses                                                     2
Internal Auditor  Fortune **** Oil and Gas  ****K                    2
Linux Technical Support Engineer (Level 1)  Leading UK Datacentre    1
Marketing and Communications Manager                                 1
                                                                    ..
Recruitment Consultant, Financial Services, Times Top 100            1
HEAD PASTRY CHEF  NORTHAMPTONSHIRE  **** ROSETTE HOTEL  ****         1
Research & Insight Analyst                                           1
Data Analyst High Wycombe up to ****/h                               1
Graduate SoftwareWeb Developer                                       1
Name: Title, Length: 50700, dtype: int64

In [6]:
# Checking value counts of the 'Title' column
Company_counts = df['Company'].value_counts()
Company_counts

non-specified                            2828
                                         1408
-                                        1095
UKStaffsearch                             434
Randstad                                  368
                                         ... 
Tag Recruitment Ltd                         1
Wave Advertising ltd                        1
Easy Web Recruitment   BSi                  1
emis                                        1
UNKNOWN   auto added by vacany import       1
Name: Company, Length: 8673, dtype: int64

In [7]:
# Correct the 'Company' column by replacing '-' and blank values with 'non-specified'
Company_counts = df['Company'].replace({'-': 'non-specified', '': 'non-specified', None: 'non-specified'})
Company_counts

0                                              CBSbutler
1        Hays Specialist Recruitment   Further Education
2                       Bennett and Game Recruitment LTD
3                              On Target Recruitment Ltd
4                        Baltic Recruitment Services Ltd
                              ...                       
50698                               Finegreen Associates
50699                                     W5 Recruitment
50700                     Executive Recruitment Services
50701                               Pursuit Training Ltd
50702              UNKNOWN   auto added by vacany import
Name: Company, Length: 50703, dtype: object

In [8]:
# Creating separate error entries for dash and blank replacements in 'Company'
errors_company = [
    {
        "indexOfdf": "ALL",
        "Id": "ALL",
        "ColumnName": "Company",
        "Original": "-",
        "Modified": "non-specified",
        "ErrorType": "Invalid Formatting",
        "Fixing": "Replaced '-' with 'non-specified'"
    },
    {
        "indexOfdf": "ALL",
        "Id": "ALL",
        "ColumnName": "Company",
        "Original": "(blank)",
        "Modified": "non-specified",
        "ErrorType": "Invalid Formatting",
        "Fixing": "Replaced blank values with 'non-specified'"
    }
]

# Create DataFrames from the error entries for 'Company'
new_errors_company_df = pd.DataFrame(errors_company)

# Use pandas.concat to add these entries to the existing error list DataFrame
error_list_df = pd.concat([error_list_df, new_errors_company_df], ignore_index=True)

error_list_df


,indexOfdf,Id,ColumnName,Original,Modified,ErrorType,Fixing
0,ALL,ALL,Company,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
1,ALL,ALL,Company,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'


In [9]:
Contract_Type = df['ContractType'].value_counts()
Contract_Type

non-specified    29089
full_time        11754
                  5839
-                 2506
part_time         1515
Name: ContractType, dtype: int64

In [10]:
# Directly replace blank strings using a more explicit method
df['ContractType'] = df['ContractType'].apply(lambda x: 'non-specified' if x.strip() == '' else x)
# Ensure all blank and '-' are correctly replaced with 'non-specified'
df['ContractType'] = df['ContractType'].replace({'-': 'non-specified', '': 'non-specified'})

# Recalculate the value counts for 'ContractType'
final_contract_type_counts = df['ContractType'].value_counts()
final_contract_type_counts

non-specified    37434
full_time        11754
part_time         1515
Name: ContractType, dtype: int64

In [11]:
# Creating separate error entries for dash and blank replacements
error_entries = [
    {
        "indexOfdf": "ALL",
        "Id": "ALL",
        "ColumnName": "ContractType",
        "Original": "-",
        "Modified": "non-specified",
        "ErrorType": "Invalid Formatting",
        "Fixing": "Replaced '-' with 'non-specified'"
    },
    {
        "indexOfdf": "ALL",
        "Id": "ALL",
        "ColumnName": "ContractType",
        "Original": "(blank)",
        "Modified": "non-specified",
        "ErrorType": "Invalid Formatting",
        "Fixing": "Replaced blank values with 'non-specified'"
    }
]

# Create DataFrames from the error entries
error_entries_df = pd.DataFrame(error_entries)

# Use pandas.concat to add these entries to the existing error list DataFrame, dropping any previous duplicates
error_list_df = pd.concat([error_list_df, error_entries_df], ignore_index=True).drop_duplicates()

error_list_df

,indexOfdf,Id,ColumnName,Original,Modified,ErrorType,Fixing
0,ALL,ALL,Company,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
1,ALL,ALL,Company,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
2,ALL,ALL,ContractType,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
3,ALL,ALL,ContractType,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'


In [12]:
Contract_Time = df['ContractTime'].value_counts()
Contract_Time

permanent        30323
non-specified    12722
contract          5579
                  1487
-                  592
Name: ContractTime, dtype: int64

In [13]:
Contract_Time = df['ContractTime'].replace({'-': 'non-specified', '': 'non-specified', None: 'non-specified'})
Contract_Time

0        permanent
1        permanent
2        permanent
3        permanent
4        permanent
           ...    
50698     contract
50699    permanent
50700    permanent
50701    permanent
50702             
Name: ContractTime, Length: 50703, dtype: object

In [14]:
# Creating separate error entries for dash and blank replacements in 'ContractTime'
errors_contract_time = [
    {
        "indexOfdf": "ALL",
        "Id": "ALL",
        "ColumnName": "ContractTime",
        "Original": "-",
        "Modified": "non-specified",
        "ErrorType": "Invalid Formatting",
        "Fixing": "Replaced '-' with 'non-specified'"
    },
    {
        "indexOfdf": "ALL",
        "Id": "ALL",
        "ColumnName": "ContractTime",
        "Original": "(blank)",
        "Modified": "non-specified",
        "ErrorType": "Invalid Formatting",
        "Fixing": "Replaced blank values with 'non-specified'"
    }
]

# Create a DataFrame from the error entries for 'ContractTime'
new_errors_contract_time_df = pd.DataFrame(errors_contract_time)

# Use pandas.concat to add these entries to the existing error list DataFrame
error_list_df = pd.concat([error_list_df, new_errors_contract_time_df], ignore_index=True)

error_list_df


,indexOfdf,Id,ColumnName,Original,Modified,ErrorType,Fixing
0,ALL,ALL,Company,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
1,ALL,ALL,Company,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
2,ALL,ALL,ContractType,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
3,ALL,ALL,ContractType,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
4,ALL,ALL,ContractTime,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
5,ALL,ALL,ContractTime,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'


In [15]:
# Check the current formatting of 'OpenDate' and 'CloseDate' columns by displaying a sample
df[['OpenDate', 'CloseDate']].head()

,OpenDate,CloseDate
0,20130301T000000,20130530T000000
1,20120804T120000,20120903T120000
2,20130224T150000,20130525T150000
3,20130222T150000,20130423T150000
4,20131012T120000,20131111T120000


In [16]:
# Convert OpenDate and CloseDate from YYYYMMDDThhmmss to datetime format yyyy-mm-dd hh:mm:ss
df['OpenDate'] = pd.to_datetime(df['OpenDate'], format='%Y%m%dT%H%M%S', errors='coerce')
df['CloseDate'] = pd.to_datetime(df['CloseDate'], format='%Y%m%dT%H%M%S', errors='coerce')


In [17]:
# Add the date format correction for 'OpenDate' and 'CloseDate' to the error list
errors_date_format = [
    {
        "indexOfdf": "ALL",
        "Id": "ALL",
        "ColumnName": "OpenDate",
        "Original": "yyyyMMddTHHmmss",
        "Modified": "yyyy-mm-dd hh:mm:ss",
        "ErrorType": "Incorrect Format",
        "Fixing": "Converted date strings from 'yyyyMMddTHHmmss' to 'yyyy-mm-dd hh:mm:ss'"
    }
]

# Create a DataFrame from the list of error entries for date format
new_errors_date_format_df = pd.DataFrame(errors_date_format)

# Use pandas.concat to add these entries to the existing error list DataFrame
error_list_df = pd.concat([error_list_df, new_errors_date_format_df], ignore_index=True)
error_list_df

,indexOfdf,Id,ColumnName,Original,Modified,ErrorType,Fixing
0,ALL,ALL,Company,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
1,ALL,ALL,Company,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
2,ALL,ALL,ContractType,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
3,ALL,ALL,ContractType,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
4,ALL,ALL,ContractTime,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
5,ALL,ALL,ContractTime,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
6,ALL,ALL,OpenDate,yyyyMMddTHHmmss,yyyy-mm-dd hh:mm:ss,Incorrect Format,Converted date strings from 'yyyyMMddTHHmmss' ...


In [18]:
errors_date_format = [
    {
        "indexOfdf": "ALL",
        "Id": "ALL",
        "ColumnName": "CloseDate",
        "Original": "yyyyMMddTHHmmss",
        "Modified": "yyyy-mm-dd hh:mm:ss",
        "ErrorType": "Incorrect Format",
        "Fixing": "Converted date strings from 'yyyyMMddTHHmmss' to 'yyyy-mm-dd hh:mm:ss'"
    }
]

# Create a DataFrame from the list of error entries for date format
new_errors_date_format_df = pd.DataFrame(errors_date_format)

# Use pandas.concat to add these entries to the existing error list DataFrame
error_list_df = pd.concat([error_list_df, new_errors_date_format_df], ignore_index=True)
error_list_df

,indexOfdf,Id,ColumnName,Original,Modified,ErrorType,Fixing
0,ALL,ALL,Company,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
1,ALL,ALL,Company,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
2,ALL,ALL,ContractType,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
3,ALL,ALL,ContractType,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
4,ALL,ALL,ContractTime,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
5,ALL,ALL,ContractTime,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
6,ALL,ALL,OpenDate,yyyyMMddTHHmmss,yyyy-mm-dd hh:mm:ss,Incorrect Format,Converted date strings from 'yyyyMMddTHHmmss' ...
7,ALL,ALL,CloseDate,yyyyMMddTHHmmss,yyyy-mm-dd hh:mm:ss,Incorrect Format,Converted date strings from 'yyyyMMddTHHmmss' ...


In [19]:
# Perform value counts for the 'SourceName' column to see the distribution and potential issues
source_name_value_counts = df['SourceName'].value_counts()
source_name_value_counts.head(20) # Display the top 20 to observe any noticeable issues

cv-library.co.uk    50703
Name: SourceName, dtype: int64

In [20]:
# Perform value counts for the 'Category' column to identify any values not matching the specified categories
category_value_counts = df['Category'].value_counts()
category_value_counts

IT Jobs                             13132
Healthcare & Nursing Jobs            8184
Engineering Jobs                     7201
Accounting & Finance Jobs            6802
Sales Jobs                           4744
Hospitality & Catering Jobs          4530
Teaching Jobs                        3558
PR, Advertising & Marketing Jobs     2552
Name: Category, dtype: int64

In [21]:
# Check value counts of the 'Location' column
df['Location'] = df['Location'].str.lower()
location_counts = df['Location'].value_counts()
location_counts


uk                   7622
london               6610
south east london    2687
the city             1129
central london        830
                     ... 
cembridge               2
livepool                2
manchaster              2
oxfords                 1
leads                   1
Name: Location, Length: 484, dtype: int64

In [22]:
# Display locations that appear infrequently, which might suggest potential misspellings or errors
infrequent_locations = location_counts[location_counts <= 5]
infrequent_locations

cembridge     2
livepool      2
manchaster    2
oxfords       1
leads         1
Name: Location, dtype: int64

In [23]:
# Example location_counts needs to be defined; for demonstration, let's use df['Location'] to create it
location_counts = df['Location'].value_counts()

# Set of common location names for reference
common_locations = set(location_counts[location_counts > 10].index)

# Misspelled locations identified
misspelled_locations = {
    "cembridge": None,
    "livepool": None,
    "manchaster": None,
    "oxfords": None,
    "leads": None
}

# Finding the closest matches for the misspelled locations
for misspelled, correct in misspelled_locations.items():
    matches = difflib.get_close_matches(misspelled, common_locations, n=1, cutoff=0.8)
    misspelled_locations[misspelled] = matches[0] if matches else misspelled  # Use the found match or keep the original if no match found

misspelled_locations

{'cembridge': 'cambridge',
 'livepool': 'liverpool',
 'manchaster': 'manchester',
 'oxfords': 'oxford',
 'leads': 'leeds'}

In [24]:
# Update the DataFrame with the correct spellings and prepare detailed error entries
detailed_error_entries = []

for misspelled, correct in misspelled_locations.items():
    # Find indices and IDs for rows with the misspelled location
    indices = df.index[df['Location'].str.lower() == misspelled].tolist()
    ids = df.loc[indices, 'Id'].tolist()
    
    # Update the DataFrame locations
    df.loc[indices, 'Location'] = correct
    
    # Append detailed error entries
    for index, id_ in zip(indices, ids):
        detailed_error_entries.append({
            "indexOfdf": index,
            "Id": id_,
            "ColumnName": "Location",
            "Original": misspelled,
            "Modified": correct,
            "ErrorType": "Misspelling",
            "Fixing": f"Replaced '{misspelled}' with '{correct}'"
        })

# Convert the detailed error list to DataFrame and append to the existing error list
detailed_error_entries_df = pd.DataFrame(detailed_error_entries)
error_list_df = pd.concat([error_list_df, detailed_error_entries_df], ignore_index=True)

# Return updated error list DataFrame and check a few entries
error_list_df


,indexOfdf,Id,ColumnName,Original,Modified,ErrorType,Fixing
0,ALL,ALL,Company,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
1,ALL,ALL,Company,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
2,ALL,ALL,ContractType,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
3,ALL,ALL,ContractType,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
4,ALL,ALL,ContractTime,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
5,ALL,ALL,ContractTime,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
6,ALL,ALL,OpenDate,yyyyMMddTHHmmss,yyyy-mm-dd hh:mm:ss,Incorrect Format,Converted date strings from 'yyyyMMddTHHmmss' ...
7,ALL,ALL,CloseDate,yyyyMMddTHHmmss,yyyy-mm-dd hh:mm:ss,Incorrect Format,Converted date strings from 'yyyyMMddTHHmmss' ...
8,8504,69621965,Location,cembridge,cambridge,Misspelling,Replaced 'cembridge' with 'cambridge'
9,45492,68018090,Location,cembridge,cambridge,Misspelling,Replaced 'cembridge' with 'cambridge'


In [25]:
non_digit_chars = df['Salary'].apply(lambda x: re.findall(r'\D+', x))
non_digit_chars.astype('str').value_counts() # as each element in non_digit_chars is a list, 
                                             # we need to convert it to string before we can do value counts

[]                    47965
['N/A']                 930
['non-specified']       453
[' ']                   386
['.']                   270
['-']                   244
[' To ']                100
[' per Annum']          100
['/Year']               100
[' - ']                 100
['K']                    50
['.', ' per hour']        3
['.', ' p/h']             2
Name: Salary, dtype: int64

In [26]:
# Reconvert 'Salary' to numeric and clean up placeholders
df['Salary_numeric'] = pd.to_numeric(df['Salary'], errors='coerce')
df['Salary_numeric'].replace({'non-specified': np.nan, 'N/A': np.nan, ' ': np.nan}, inplace=True)

# Encode categorical variables
le_category = LabelEncoder()
le_contract_time = LabelEncoder()
df['Category_encoded'] = le_category.fit_transform(df['Category'])
df['ContractTime_encoded'] = le_contract_time.fit_transform(df['ContractTime'])

# Selecting predictors and target
predictors = ['Category_encoded', 'ContractTime_encoded']
X = df[predictors]
y = df['Salary_numeric']

# Filter out rows with NaN in 'Salary_numeric' for training
X_train = X[y.notna()]
y_train = y[y.notna()]

# Training the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Impute missing salaries
for idx in df[df['Salary_numeric'].isna()].index:
    features = df.loc[idx, predictors].values.reshape(1, -1)
    df.at[idx, 'Salary_numeric'] = model.predict(features)[0]

# Display updated DataFrame with filled in salaries
df[['Id', 'Title', 'Salary', 'Salary_numeric']].head(20)


,Id,Title,Salary,Salary_numeric
0,70215060,Sales Bid Coordinator,24000,24000.0
1,71749951,"Primary Teachers Nailsea, Yatton, Congresbury",18720,18720.0
2,69080174,Internal Technical Engineer Heat Pumps,19000,19000.0
3,67638009,Regional Sales Manager : PPE and Gas Detectors,42500,42500.0
4,68217496,Senior Assembly Engineer,38750,38750.0
5,69967015,Associate Civil Infrastructure Design Engineer,45000,45000.0
6,66600487,Sales Advisor **** an hour Immediate Start,13440,13440.0
7,66925309,"Service Desk Analyst Active Directory, Citrix...",23500,23500.0
8,69001320,Field Sales (Home based),27500,27500.0
9,70216327,Risk Manager (Junior/Senior),45000,45000.0


In [27]:
# Creating separate error entries for each type of placeholder in 'Salary'
errors_salary_separate = [
    {
        "indexOfdf": "ALL",
        "Id": "ALL",
        "ColumnName": "Salary",
        "Original": "non-specified",
        "Modified": "Filled Values Using Linear Model",
        "ErrorType": "Data Cleaning",
        "Fixing": "Converted 'non-specified' to NaN"
    },
    {
        "indexOfdf": "ALL",
        "Id": "ALL",
        "ColumnName": "Salary",
        "Original": "N/A",
        "Modified": "Filled Values Using Linear Model",
        "ErrorType": "Data Cleaning",
        "Fixing": "Converted 'N/A' to NaN"
    },
    {
        "indexOfdf": "ALL",
        "Id": "ALL",
        "ColumnName": "Salary",
        "Original": "(blank)",
        "Modified": "Filled Values Using Linear Model",
        "ErrorType": "Data Cleaning",
        "Fixing": "Converted blank values to NaN"
    }
]

# Create a DataFrame from the separate error entries for 'Salary'
new_errors_salary_separate_df = pd.DataFrame(errors_salary_separate)

# Use pandas.concat to add these entries to the existing error list DataFrame
error_list_df = pd.concat([error_list_df, new_errors_salary_separate_df], ignore_index=True)

error_list_df


,indexOfdf,Id,ColumnName,Original,Modified,ErrorType,Fixing
0,ALL,ALL,Company,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
1,ALL,ALL,Company,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
2,ALL,ALL,ContractType,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
3,ALL,ALL,ContractType,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
4,ALL,ALL,ContractTime,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
5,ALL,ALL,ContractTime,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
6,ALL,ALL,OpenDate,yyyyMMddTHHmmss,yyyy-mm-dd hh:mm:ss,Incorrect Format,Converted date strings from 'yyyyMMddTHHmmss' ...
7,ALL,ALL,CloseDate,yyyyMMddTHHmmss,yyyy-mm-dd hh:mm:ss,Incorrect Format,Converted date strings from 'yyyyMMddTHHmmss' ...
8,8504,69621965,Location,cembridge,cambridge,Misspelling,Replaced 'cembridge' with 'cambridge'
9,45492,68018090,Location,cembridge,cambridge,Misspelling,Replaced 'cembridge' with 'cambridge'


In [28]:
# Convert 'Salary' to numeric, replace non-numeric values with NaN, round to two decimals
df['Salary'] = pd.to_numeric(df['Salary'], errors='coerce')
df['Salary'] = df['Salary'].round(2)

# Display the DataFrame with the corrected 'Salary' column
df[['Id', 'Title', 'Salary']].head(20)

,Id,Title,Salary
0,70215060,Sales Bid Coordinator,24000.0
1,71749951,"Primary Teachers Nailsea, Yatton, Congresbury",18720.0
2,69080174,Internal Technical Engineer Heat Pumps,19000.0
3,67638009,Regional Sales Manager : PPE and Gas Detectors,42500.0
4,68217496,Senior Assembly Engineer,38750.0
5,69967015,Associate Civil Infrastructure Design Engineer,45000.0
6,66600487,Sales Advisor **** an hour Immediate Start,13440.0
7,66925309,"Service Desk Analyst Active Directory, Citrix...",23500.0
8,69001320,Field Sales (Home based),27500.0
9,70216327,Risk Manager (Junior/Senior),45000.0


In [29]:
# Drop multiple columns
df = df.drop(['Salary_numeric', 'Category_encoded', 'ContractTime_encoded'], axis=1)


In [30]:
df.columns

Index(['Id', 'Title', 'Location', 'Company', 'ContractType', 'ContractTime',
       'Category', 'Salary', 'OpenDate', 'CloseDate', 'SourceName'],
      dtype='object')

In [31]:
# Apply the transformations to the 'Salary' column in the DataFrame 'df'
df['Salary'] = df['Salary'].apply(lambda x: re.sub(',', '', str(x)))
df['Salary'] = df['Salary'].apply(lambda x: re.sub('k', '000', x, flags=re.IGNORECASE))

# Display the head of the DataFrame to check the result after modifications
df['Salary'].head(10)


0    24000.0
1    18720.0
2    19000.0
3    42500.0
4    38750.0
5    45000.0
6    13440.0
7    23500.0
8    27500.0
9    45000.0
Name: Salary, dtype: object

In [32]:
# Define the new error entries for the Salary column transformations
new_errors_salary = [
    {
        "indexOfdf": "ALL",
        "Id": "ALL",
        "ColumnName": "Salary",
        "Original": ",",
        "Modified": "removed comma",
        "ErrorType": "Formatting Issue",
        "Fixing": "Removed commas from salary values"
    },
    {
        "indexOfdf": "ALL",
        "Id": "ALL",
        "ColumnName": "Salary",
        "Original": "k",
        "Modified": "000",
        "ErrorType": "Abbreviation",
        "Fixing": "Replaced 'k' with '000' in salary values"
    }
]

# Create a DataFrame from the list of new error entries for Salary
new_errors_salary_df = pd.DataFrame(new_errors_salary)

# Use pandas.concat to add these entries to the existing error list DataFrame
error_list_df = pd.concat([error_list_df, new_errors_salary_df], ignore_index=True)

# Display the updated error list DataFrame
error_list_df


,indexOfdf,Id,ColumnName,Original,Modified,ErrorType,Fixing
0,ALL,ALL,Company,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
1,ALL,ALL,Company,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
2,ALL,ALL,ContractType,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
3,ALL,ALL,ContractType,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
4,ALL,ALL,ContractTime,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
5,ALL,ALL,ContractTime,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
6,ALL,ALL,OpenDate,yyyyMMddTHHmmss,yyyy-mm-dd hh:mm:ss,Incorrect Format,Converted date strings from 'yyyyMMddTHHmmss' ...
7,ALL,ALL,CloseDate,yyyyMMddTHHmmss,yyyy-mm-dd hh:mm:ss,Incorrect Format,Converted date strings from 'yyyyMMddTHHmmss' ...
8,8504,69621965,Location,cembridge,cambridge,Misspelling,Replaced 'cembridge' with 'cambridge'
9,45492,68018090,Location,cembridge,cambridge,Misspelling,Replaced 'cembridge' with 'cambridge'


In [33]:
def convert_salary_ranges(salary):
    # Detect and convert salary ranges to their mean
    if ' - ' in salary or ' To ' in salary:
        parts = re.split(r' - | To ', salary)
        if all(part.isdigit() for part in parts):
            numeric_parts = list(map(int, parts))
            return sum(numeric_parts) / len(numeric_parts)
    return salary  # Return original if not a range

# Apply the function to the 'Salary' column
df['Salary'] = df['Salary'].apply(lambda x: convert_salary_ranges(str(x)))

# Display updated DataFrame
df['Salary'].head()


0    24000.0
1    18720.0
2    19000.0
3    42500.0
4    38750.0
Name: Salary, dtype: object

In [34]:
# Define the error entry for processing salary ranges with 'To' separator
error_entry_salary_range_to = {
    "indexOfdf": "ALL",
    "Id": "ALL",
    "ColumnName": "Salary",
    "Original": "Salary ranges (e.g., '25000 To 30000')",
    "Modified": "Mean of range",
    "ErrorType": "Salary Range Conversion (To Separator)",
    "Fixing": "Converted salary ranges with 'To' separator to their mean values"
}

# Define the error entry for processing salary ranges with '-' separator
error_entry_salary_range_dash = {
    "indexOfdf": "ALL",
    "Id": "ALL",
    "ColumnName": "Salary",
    "Original": "Salary ranges (e.g., '30000 - 35000')",
    "Modified": "Mean of range",
    "ErrorType": "Salary Range Conversion (Dash Separator)",
    "Fixing": "Converted salary ranges with '-' separator to their mean values"
}

# Create DataFrames from the error entries
new_error_salary_range_to_df = pd.DataFrame([error_entry_salary_range_to])
new_error_salary_range_dash_df = pd.DataFrame([error_entry_salary_range_dash])

# Append the new error entries to the existing error list DataFrame
error_list_df = pd.concat([error_list_df, new_error_salary_range_to_df, new_error_salary_range_dash_df], ignore_index=True)

# Display the updated error list DataFrame
error_list_df


,indexOfdf,Id,ColumnName,Original,Modified,ErrorType,Fixing
0,ALL,ALL,Company,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
1,ALL,ALL,Company,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
2,ALL,ALL,ContractType,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
3,ALL,ALL,ContractType,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
4,ALL,ALL,ContractTime,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
5,ALL,ALL,ContractTime,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
6,ALL,ALL,OpenDate,yyyyMMddTHHmmss,yyyy-mm-dd hh:mm:ss,Incorrect Format,Converted date strings from 'yyyyMMddTHHmmss' ...
7,ALL,ALL,CloseDate,yyyyMMddTHHmmss,yyyy-mm-dd hh:mm:ss,Incorrect Format,Converted date strings from 'yyyyMMddTHHmmss' ...
8,8504,69621965,Location,cembridge,cambridge,Misspelling,Replaced 'cembridge' with 'cambridge'
9,45492,68018090,Location,cembridge,cambridge,Misspelling,Replaced 'cembridge' with 'cambridge'


In [35]:
# Function to convert salary strings to annual salaries if specified on an hourly basis
def convert_to_annual_salary(salary):
    # Standard conversion factor for hourly to yearly
    hours_per_week = 40
    weeks_per_year = 52
    annual_hours = hours_per_week * weeks_per_year

    # Check if salary is already numeric (float or int)
    if isinstance(salary, (float, int)):
        return salary

    # Remove commas for processing, and find digits and possible decimal points
    salary = str(salary).replace(',', '').strip()

    # Check for different hourly indications
    if any(x in salary.lower() for x in ['per hour', 'p/h']):
        # Extract the numeric part of the salary
        hourly_rate = re.findall(r'\d+\.?\d*', salary)
        if hourly_rate:
            # Convert the first found rate (assuming the first is the rate)
            annual_salary = float(hourly_rate[0]) * annual_hours
            return str(int(annual_salary))

    # Check for annual indicators
    elif any(x in salary.lower() for x in ['per annum', 'per year', '/year']):
        # Simply clean up and return the first found numeric annual rate
        annual_rate = re.findall(r'\d+\.?\d*', salary)
        if annual_rate:
            return str(int(float(annual_rate[0])))

    # Return the original salary if no conversions are needed
    return salary

# Apply the conversion function to the Salary column
df['Salary'] = df['Salary'].apply(convert_to_annual_salary)

# Now we'll see the updated data with converted salaries
salary_converted_counts = df['Salary'].value_counts()

salary_converted_counts


nan        2468
35000.0    1740
30000.0    1685
40000.0    1543
45000.0    1357
           ... 
37692.0       1
19150.0       1
28100.0       1
15375.0       1
71040.0       1
Name: Salary, Length: 3110, dtype: int64

In [36]:
# Error records for hourly to annual conversion
error_record_hourly_to_annual = {
    "indexOfdf": "ALL",
    "Id": "ALL",
    "ColumnName": "Salary",
    "Original": "Hourly rate (e.g., per hour, p/h)",
    "Modified": "Converted to annual salary based on standard hours per year",
    "ErrorType": "Conversion to Annual Salary",
    "Fixing": "Applied conversion from hourly rate to annual salary using standard work hours"
}

# Error records for annual rates validation
error_record_annual_rate = {
    "indexOfdf": "ALL",
    "Id": "ALL",
    "ColumnName": "Salary",
    "Original": "Annual rate (e.g., per annum, per year, /year)",
    "Modified": "Confirmed as annual salary without conversion",
    "ErrorType": "Validation of Annual Salary",
    "Fixing": "Confirmed the format and cleaned any non-numeric characters"
}

# Error records for retention of original values
error_record_retention = {
    "indexOfdf": "ALL",
    "Id": "ALL",
    "ColumnName": "Salary",
    "Original": "Original salary format not requiring conversion",
    "Modified": "Retained original salary without changes",
    "ErrorType": "No Conversion Required",
    "Fixing": "No conversion applied, original salary format retained"
}
# Create DataFrames from the error records
error_record_hourly_to_annual_df = pd.DataFrame([error_record_hourly_to_annual])
error_record_annual_rate_df = pd.DataFrame([error_record_annual_rate])

# Use pandas.concat to add these entries to the existing error list DataFrame
error_list_df = pd.concat([error_list_df, 
                           error_record_hourly_to_annual_df, 
                           error_record_annual_rate_df], ignore_index=True)
# Display the updated error list DataFrame
error_list_df



,indexOfdf,Id,ColumnName,Original,Modified,ErrorType,Fixing
0,ALL,ALL,Company,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
1,ALL,ALL,Company,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
2,ALL,ALL,ContractType,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
3,ALL,ALL,ContractType,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
4,ALL,ALL,ContractTime,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
5,ALL,ALL,ContractTime,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
6,ALL,ALL,OpenDate,yyyyMMddTHHmmss,yyyy-mm-dd hh:mm:ss,Incorrect Format,Converted date strings from 'yyyyMMddTHHmmss' ...
7,ALL,ALL,CloseDate,yyyyMMddTHHmmss,yyyy-mm-dd hh:mm:ss,Incorrect Format,Converted date strings from 'yyyyMMddTHHmmss' ...
8,8504,69621965,Location,cembridge,cambridge,Misspelling,Replaced 'cembridge' with 'cambridge'
9,45492,68018090,Location,cembridge,cambridge,Misspelling,Replaced 'cembridge' with 'cambridge'


In [37]:
def check_date_violation(open_date, close_date):
    # First, check if either date is None or not a string
    if not isinstance(open_date, str) or not isinstance(close_date, str):
        return 0  # No violation if there's missing data
    
    # Try to extract the year from the open date and close date
    open_match = re.search(r'\d{4}', open_date)
    close_match = re.search(r'\d{4}', close_date)
    
    if open_match and close_match:
        open_year = int(open_match.group(0))
        close_year = int(close_match.group(0))
        return int(open_year > close_year)  # True if open year is greater than close year
    return 0  # No violation if years can't be extracted

# Apply the function to the dataset
df['violated'] = df.apply(lambda row: check_date_violation(row['OpenDate'], row['CloseDate']), axis=1)

# Display the 'violated' column to check the result
df['violated']


0        0
1        0
2        0
3        0
4        0
        ..
50698    0
50699    0
50700    0
50701    0
50702    0
Name: violated, Length: 50703, dtype: int64

In [38]:
missing_counts = df.isnull().sum()
print(missing_counts)

Id              0
Title           0
Location        0
Company         0
ContractType    0
ContractTime    0
Category        0
Salary          0
OpenDate        1
CloseDate       0
SourceName      0
violated        0
dtype: int64


In [39]:
most_common_date = df['OpenDate'].mode()[0]
df['OpenDate'] = df['OpenDate'].fillna(most_common_date)

In [40]:
# General Error Record for Missing 'OpenDate'
error_record_missing_open_date = {
    "indexOfdf": "ALL",
    "Id": "ALL",
    "ColumnName": "OpenDate",
    "Original": "Missing data",
    "Modified": "To be filled or row to be removed",
    "ErrorType": "Missing Data",
    "Fixing": "Suggested actions: Fill with a specific date, use the most common date, or remove the row"
}
# Create DataFrame from the error record
error_record_missing_open_date_df = pd.DataFrame([error_record_missing_open_date])

# Append the new error record to the existing DataFrame
error_list_df = pd.concat([error_list_df, error_record_missing_open_date_df], ignore_index=True)

error_list_df

,indexOfdf,Id,ColumnName,Original,Modified,ErrorType,Fixing
0,ALL,ALL,Company,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
1,ALL,ALL,Company,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
2,ALL,ALL,ContractType,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
3,ALL,ALL,ContractType,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
4,ALL,ALL,ContractTime,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
5,ALL,ALL,ContractTime,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
6,ALL,ALL,OpenDate,yyyyMMddTHHmmss,yyyy-mm-dd hh:mm:ss,Incorrect Format,Converted date strings from 'yyyyMMddTHHmmss' ...
7,ALL,ALL,CloseDate,yyyyMMddTHHmmss,yyyy-mm-dd hh:mm:ss,Incorrect Format,Converted date strings from 'yyyyMMddTHHmmss' ...
8,8504,69621965,Location,cembridge,cambridge,Misspelling,Replaced 'cembridge' with 'cambridge'
9,45492,68018090,Location,cembridge,cambridge,Misspelling,Replaced 'cembridge' with 'cambridge'


In [41]:
df.columns

Index(['Id', 'Title', 'Location', 'Company', 'ContractType', 'ContractTime',
       'Category', 'Salary', 'OpenDate', 'CloseDate', 'SourceName',
       'violated'],
      dtype='object')

In [42]:
df = df.drop(['violated'], axis=1)
df.head(n=15)

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,70215060,Sales Bid Coordinator,surrey,CBSbutler,non-specified,permanent,Engineering Jobs,24000.0,2013-03-01 00:00:00,2013-05-30 00:00:00,cv-library.co.uk
1,71749951,"Primary Teachers Nailsea, Yatton, Congresbury",weston-super-mare,Hays Specialist Recruitment Further Education,non-specified,permanent,Teaching Jobs,18720.0,2012-08-04 12:00:00,2012-09-03 12:00:00,cv-library.co.uk
2,69080174,Internal Technical Engineer Heat Pumps,derbyshire,Bennett and Game Recruitment LTD,non-specified,permanent,Engineering Jobs,19000.0,2013-02-24 15:00:00,2013-05-25 15:00:00,cv-library.co.uk
3,67638009,Regional Sales Manager : PPE and Gas Detectors,uk,On Target Recruitment Ltd,non-specified,permanent,Sales Jobs,42500.0,2013-02-22 15:00:00,2013-04-23 15:00:00,cv-library.co.uk
4,68217496,Senior Assembly Engineer,durham,Baltic Recruitment Services Ltd,non-specified,permanent,Engineering Jobs,38750.0,2013-10-12 12:00:00,2013-11-11 12:00:00,cv-library.co.uk
5,69967015,Associate Civil Infrastructure Design Engineer,essex,Energi Recruitment Services Ltd,non-specified,permanent,Engineering Jobs,45000.0,2013-11-28 00:00:00,2013-12-28 00:00:00,cv-library.co.uk
6,66600487,Sales Advisor **** an hour Immediate Start,sittingbourne,Huntress Group,non-specified,contract,Sales Jobs,13440.0,2012-05-08 15:00:00,2012-06-07 15:00:00,cv-library.co.uk
7,66925309,"Service Desk Analyst Active Directory, Citrix...",london,Pearson Whiffin Affinity,non-specified,permanent,IT Jobs,23500.0,2012-10-03 15:00:00,2013-01-01 15:00:00,cv-library.co.uk
8,69001320,Field Sales (Home based),birmingham,Parkside,non-specified,permanent,Sales Jobs,27500.0,2013-03-13 00:00:00,2013-06-11 00:00:00,cv-library.co.uk
9,70216327,Risk Manager (Junior/Senior),bristol,Electus Recruitment Solutions,non-specified,permanent,Engineering Jobs,45000.0,2012-10-09 15:00:00,2012-12-08 15:00:00,cv-library.co.uk


In [43]:
#Converting the modified xml file and error list to csv
df.to_csv('s3994666_dataset1_solution.csv', index=False)
error_list_df.to_csv('s3994666_errorlist.csv', index=False)

In [44]:
df = pd.read_csv('s3994666_dataset1_solution.csv')
df.head(n=20)

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,70215060,Sales Bid Coordinator,surrey,CBSbutler,non-specified,permanent,Engineering Jobs,24000.0,2013-03-01 00:00:00,2013-05-30 00:00:00,cv-library.co.uk
1,71749951,"Primary Teachers Nailsea, Yatton, Congresbury",weston-super-mare,Hays Specialist Recruitment Further Education,non-specified,permanent,Teaching Jobs,18720.0,2012-08-04 12:00:00,2012-09-03 12:00:00,cv-library.co.uk
2,69080174,Internal Technical Engineer Heat Pumps,derbyshire,Bennett and Game Recruitment LTD,non-specified,permanent,Engineering Jobs,19000.0,2013-02-24 15:00:00,2013-05-25 15:00:00,cv-library.co.uk
3,67638009,Regional Sales Manager : PPE and Gas Detectors,uk,On Target Recruitment Ltd,non-specified,permanent,Sales Jobs,42500.0,2013-02-22 15:00:00,2013-04-23 15:00:00,cv-library.co.uk
4,68217496,Senior Assembly Engineer,durham,Baltic Recruitment Services Ltd,non-specified,permanent,Engineering Jobs,38750.0,2013-10-12 12:00:00,2013-11-11 12:00:00,cv-library.co.uk
5,69967015,Associate Civil Infrastructure Design Engineer,essex,Energi Recruitment Services Ltd,non-specified,permanent,Engineering Jobs,45000.0,2013-11-28 00:00:00,2013-12-28 00:00:00,cv-library.co.uk
6,66600487,Sales Advisor **** an hour Immediate Start,sittingbourne,Huntress Group,non-specified,contract,Sales Jobs,13440.0,2012-05-08 15:00:00,2012-06-07 15:00:00,cv-library.co.uk
7,66925309,"Service Desk Analyst Active Directory, Citrix...",london,Pearson Whiffin Affinity,non-specified,permanent,IT Jobs,23500.0,2012-10-03 15:00:00,2013-01-01 15:00:00,cv-library.co.uk
8,69001320,Field Sales (Home based),birmingham,Parkside,non-specified,permanent,Sales Jobs,27500.0,2013-03-13 00:00:00,2013-06-11 00:00:00,cv-library.co.uk
9,70216327,Risk Manager (Junior/Senior),bristol,Electus Recruitment Solutions,non-specified,permanent,Engineering Jobs,45000.0,2012-10-09 15:00:00,2012-12-08 15:00:00,cv-library.co.uk


In [46]:
df = pd.read_csv('s3994666_errorlist.csv')
df

,indexOfdf,Id,ColumnName,Original,Modified,ErrorType,Fixing
0,ALL,ALL,Company,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
1,ALL,ALL,Company,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
2,ALL,ALL,ContractType,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
3,ALL,ALL,ContractType,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
4,ALL,ALL,ContractTime,-,non-specified,Invalid Formatting,Replaced '-' with 'non-specified'
5,ALL,ALL,ContractTime,(blank),non-specified,Invalid Formatting,Replaced blank values with 'non-specified'
6,ALL,ALL,OpenDate,yyyyMMddTHHmmss,yyyy-mm-dd hh:mm:ss,Incorrect Format,Converted date strings from 'yyyyMMddTHHmmss' ...
7,ALL,ALL,CloseDate,yyyyMMddTHHmmss,yyyy-mm-dd hh:mm:ss,Incorrect Format,Converted date strings from 'yyyyMMddTHHmmss' ...
8,8504,69621965,Location,cembridge,cambridge,Misspelling,Replaced 'cembridge' with 'cambridge'
9,45492,68018090,Location,cembridge,cambridge,Misspelling,Replaced 'cembridge' with 'cambridge'
